# Python analysis of Project Rephetio epilepsy predictions

In [1]:
import pandas
import collections

In [2]:
pk_df = (
    pandas.read_excel('data/top-5-percent-PK-plots.xlsx', skiprows=1)
    .iloc[:100, :11]
    .rename(columns={'Name': 'name', 'Prediction score': 'prediction', 'Disease Pctl': 'disease_pctl'})
)

pk_df['status'] = (pk_df
    [['AED', 'Anti-Epileptic properties', 'Induces seizure']]
    .astype(int).max(axis='columns')
    .map({4: 'AED', 3: 'AEP', 1: 'IS', 0: '?'})
)

pk_df = pk_df[['name', 'prediction', 'disease_pctl', 'status']]
pk_df.head(2)

,name,prediction,disease_pctl,status
0,Topiramate,0.603,1.000,AED
1,Ethotoin,0.589,0.999,AED


In [3]:
statuses = sorted(pk_df.status.unique())

In [4]:
def rolling_groups(df, k=5):
    """Yield rolling windows on dataframe"""
    for i in range(len(df)):
        start = max(0, i - k)
        end = i + k + 1
        yield df.iloc[start:end, :]

def summarize_window(df):
    s = pandas.Series()
    s['min_pred'] = min(df.prediction)
    s['max_pred'] = max(df.prediction)
    counter = collections.Counter(df.status)
    for status in statuses:
        s['freq_' + status] = counter[status] / len(df)
    return s

rolling_df = pandas.DataFrame.from_records(
    map(summarize_window, rolling_groups(pk_df, k=7))
)

In [5]:
plot_df = pandas.concat([pk_df, rolling_df], axis='rows')

In [6]:
plot_df.tail()

,name,prediction,disease_pctl,status,min_pred,max_pred,freq_?,freq_AED,freq_AEP,freq_IS
95,Desipramine,0.0308,0.938,IS,0.0298,0.0330,0.166667,0.416667,0.083333,0.333333
96,Dabrafenib,0.0306,0.938,?,0.0298,0.0329,0.090909,0.454545,0.090909,0.363636
97,Rufinamide,0.0305,0.937,AED,0.0298,0.0327,0.100000,0.400000,0.100000,0.400000
98,Memantine,0.0303,0.936,IS,0.0298,0.0327,0.111111,0.444444,0.000000,0.444444
99,Zolpidem,0.0298,0.936,AED,0.0298,0.0325,0.125000,0.500000,0.000000,0.375000


In [7]:
plot_df.to_csv('data/windows.tsv', sep='\t', index=False, float_format='%.5g')

## Stats

In [8]:
len(plot_df)

100

In [9]:
plot_df.status.value_counts()

AED    69
IS     15
?       9
AEP     7
Name: status, dtype: int64

In [10]:
url = 'https://github.com/dhimmel/learn/raw/d2251a942813015d0362a90f179c961016336e77/summary/indications.tsv'
aeds_in_phdb = (
    pandas.read_table(url)
    .query("rel_type == 'TREATS_CtD'")
    .query("disease_id == 'DOID:1826'")
    .compound_name
    .tolist()
)
# Number of disease-modifying antiepileptics in PharmacotherapyDB
len(set(aeds_in_phdb))

25

In [11]:
# Number of disease-modifying antiepileptics from PharmacotherapyDB in top 100 predictions
len(set(aeds_in_phdb) & set(plot_df.name))

23

In [12]:
# Disease-modifying antiepileptics from PharmacotherapyDB not in top 100 predictions
set(aeds_in_phdb) - set(plot_df.name)

{'Propofol', 'Vigabatrin'}

In [13]:
plot_df.query("name in @aeds_in_phdb")

,name,prediction,disease_pctl,status,min_pred,max_pred,freq_?,freq_AED,freq_AEP,freq_IS
0,Topiramate,0.6030,1.000,AED,0.4600,0.6030,0.000000,1.000000,0.000000,0.000000
4,Primidone,0.4940,0.997,AED,0.4020,0.6030,0.000000,1.000000,0.000000,0.000000
6,Gabapentin,0.4770,0.996,AED,0.3750,0.6030,0.000000,1.000000,0.000000,0.000000
7,Diazepam,0.4600,0.995,AED,0.3620,0.6030,0.000000,1.000000,0.000000,0.000000
10,Valproic Acid,0.4090,0.993,AED,0.3540,0.5650,0.000000,1.000000,0.000000,0.000000
14,Phenytoin,0.3620,0.991,AED,0.2840,0.4600,0.000000,1.000000,0.000000,0.000000
16,Clonazepam,0.3540,0.990,AED,0.2820,0.4300,0.000000,1.000000,0.000000,0.000000
17,Oxcarbazepine,0.3540,0.989,AED,0.2520,0.4090,0.000000,0.933333,0.000000,0.066667
18,Carbamazepine,0.3410,0.988,AED,0.2480,0.4020,0.000000,0.933333,0.000000,0.066667
19,Midazolam,0.3300,0.988,AED,0.2460,0.3810,0.000000,0.933333,0.000000,0.066667
